In [40]:
# dependencies and set up
import pandas as pd
from pathlib import Path 

In [41]:
#load the school csv
school_path=Path("Resources/schools_complete.csv")
school_csv=pd.read_csv(school_path)

In [42]:
# load the students csv 
students_path=Path("Resources/students_complete.csv")
students_csv=pd.read_csv(students_path)

In [43]:
# combine the above into 1 data frame
combined_data=pd.merge(school_csv, students_csv, how="left", on=["school_name", "school_name"])
combined_data

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84
...,...,...,...,...,...,...,...,...,...,...,...
39165,14,Thomas High School,Charter,1635,1043130,39165,Donna Howard,F,12th,99,90
39166,14,Thomas High School,Charter,1635,1043130,39166,Dawn Bell,F,10th,95,70
39167,14,Thomas High School,Charter,1635,1043130,39167,Rebecca Tanner,F,9th,73,84
39168,14,Thomas High School,Charter,1635,1043130,39168,Desiree Kidd,F,10th,99,90


In [44]:
#find the unique amount of schools
school_count=len(combined_data["school_name"].unique())
school_count

15

In [45]:
#find the total number of students
student_count=len(combined_data["Student ID"].unique())
student_count

39170

In [46]:
#calculate the budget
budget=sum(combined_data["budget"].unique())
budget

24649428

In [47]:
#calculate average math schools
maths=combined_data["math_score"].mean()
maths

78.98537145774827

In [48]:
# calucate average reading score
reading=combined_data["reading_score"].mean()
reading

81.87784018381414

In [49]:
#calculate percentage of students who passed math (scores higher or equal to 70)
passing_math_count = combined_data[(combined_data["math_score"] >=70)].count()["student_name"]
passing_math_count = passing_math_count / float(student_count) * 100
passing_math_count

74.9808526933878

In [50]:
#calculate percentage of students who passed reading (scores higher or equal to 70)
passing_reading_count = combined_data[(combined_data["reading_score"] >=70)].count()["student_name"]
passing_reading_count = passing_reading_count / float(student_count) * 100
passing_reading_count

85.80546336482001

In [51]:
# find percent that passed both
passing_both=combined_data[(combined_data["math_score"] >= 70) & (combined_data["reading_score"] >= 70)].count()["student_name"]
overall_passing= passing_both / float(student_count) * 100
overall_passing

65.17232575950983

In [52]:
# create a district level summary
district = [{"Total Schools": school_count, 
             "Total Students": student_count, 
             "Total Budget": budget, 
             "Average Math Score": round(maths, 2),
             "Average Reading Score": round(reading, 2),
             "% Students Passing Math": round(passing_math_count, 2),
             "% Students Passing Reading": round(passing_reading_count, 2),
             "Overall Passing Rate": round(overall_passing, 2)}]
district_summary = pd.DataFrame(district)

#formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

#display the dataframe
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Students Passing Math,% Students Passing Reading,Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,65.17


In [53]:
# find all the school types, set index code from geekforgeeks.com
school_types=school_csv.set_index(["school_name"])["type"]

In [54]:
# find student counts per school
per_school_counts=school_csv.set_index(["school_name"])["size"]

In [55]:
#calculate total school budget and per capita spending per school
per_school_budget = school_csv.set_index(["school_name"])["budget"]

per_school_capita = (school_csv.set_index(["school_name"])["budget"])/(school_csv.set_index(["school_name"])["size"])

In [56]:
#calculate the average per school math and reading test scores
per_school=combined_data.set_index("school_name").groupby(["school_name"])
per_school_math=per_school["math_score"].mean()
per_school_reading=per_school["reading_score"].mean()

In [57]:
# calculate the number of students per school passing reading
per_school_passreading = combined_data[combined_data["reading_score"] >= 70].groupby("school_name")["Student ID"].count()

In [58]:
# calculate the number of students per school passing math
per_school_passmath = combined_data[combined_data["math_score"] >= 70].groupby("school_name")["Student ID"].count()

In [59]:
# of students per school that passed both math and reading (scores > 70)
per_school_passing_both_data = combined_data[(combined_data["reading_score"] >= 70) & (combined_data["math_score"] >= 70)]

per_school_passing_both = per_school_passing_both_data.groupby(["school_name"]).size()

In [60]:
# calculate the passing rate percentages
per_school_passreading_percent = (per_school_passreading / per_school_counts) * 100
per_school_passmath_percent = (per_school_passreading / per_school_counts) * 100
per_school_both_percent = (per_school_passing_both / per_school_counts) * 100

In [78]:
# create a dataframe with columns for the calcuations above
per_school_summary= pd.DataFrame({"School Types": school_types, 
             "Total Students per School": per_school_counts, 
             "Total School Budget": per_school_budget, 
             "Per Student Budget": per_school_capita,
             "Average Math Score": round(per_school_math, 2),
             "Average Reading Score": round(per_school_reading, 2),
             "% Students Passing Math": round(per_school_passreading_percent, 2),
             "% Students Passing Reading": round(per_school_passmath_percent, 2),
             "% Overall Passing Rate": round(per_school_both_percent, 2)})

school_spending_df = per_school_summary.copy()

per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

per_school_summary

,School Types,Total Students per School,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Students Passing Math,% Students Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,81.93,81.93,54.64
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,97.04,97.04,91.33
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,80.74,80.74,53.20
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,79.30,79.30,54.29
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,97.14,97.14,90.60
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,80.86,80.86,53.53
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,96.25,96.25,89.23
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,81.32,81.32,53.51
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,81.22,81.22,53.54


In [62]:
# sort by % overall passing in descending order and display top 5 rows
top_schools = per_school_summary.sort_values("% Overall Passing Rate", ascending=False)
top_schools.head()

,School Types,Total Students per School,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Students Passing Math,% Students Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,97.04,97.04,91.33
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,97.31,97.31,90.95
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,97.14,97.14,90.60
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,96.54,96.54,90.58
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,95.95,95.95,90.54


In [63]:
# sort by % overall passing rate in ascending order and display top 5 rows
bottom_schools = per_school_summary.sort_values("% Overall Passing Rate", ascending=True)
bottom_schools.head()

,School Types,Total Students per School,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Students Passing Math,% Students Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,80.22,80.22,52.99
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,80.74,80.74,53.20
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,81.32,81.32,53.51
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,80.86,80.86,53.53
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,81.22,81.22,53.54


In [64]:
# separate the data by grade
ninth_graders = combined_data[(combined_data["grade"] == "9th")]
tenth_graders = combined_data[(combined_data["grade"] == "10th")]
eleventh_graders = combined_data[(combined_data["grade"] == "11th")]
twelth_graders = combined_data[(combined_data["grade"] == "12th")]

#group by school name and take the mean of the math school for each
ninth_grade_math = ninth_graders.groupby("school_name")["math_score"].mean()
tenth_grade_math = tenth_graders.groupby("school_name")["math_score"].mean()
eleventh_grade_math = eleventh_graders.groupby("school_name")["math_score"].mean()
twelth_grade_math = twelth_graders.groupby("school_name")["math_score"].mean()

# combine each of the scores into 1 DataFrame
math_scores_by_grade = pd.DataFrame({"Ninth Grade Math Scores": ninth_grade_math,
                                     "Tenth Grade Math Scores": tenth_grade_math,
                                     "Eleventh Grade Math Scores": eleventh_grade_math,
                                     "Twelth Grade Math Scores": twelth_grade_math
    
})

# data wrangling
math_scores_by_grade.index.name = None

#display DF
math_scores_by_grade

,Ninth Grade Math Scores,Tenth Grade Math Scores,Eleventh Grade Math Scores,Twelth Grade Math Scores
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


In [65]:
#group by school name and take the mean of the reading school for each
ninth_grade_reading = ninth_graders.groupby("school_name")["reading_score"].mean()
tenth_grade_reading = tenth_graders.groupby("school_name")["reading_score"].mean()
eleventh_grade_reading = eleventh_graders.groupby("school_name")["reading_score"].mean()
twelth_grade_reading = twelth_graders.groupby("school_name")["reading_score"].mean()

# combine each of the scores into 1 DataFrame
reading_scores_by_grade = pd.DataFrame({"Ninth Grade Reading Scores": ninth_grade_reading,
                                     "Tenth Grade Reading Scores": tenth_grade_reading,
                                     "Eleventh Grade Reading Scores": eleventh_grade_reading,
                                     "Twelth Grade Reading Scores": twelth_grade_reading})
    
# Minor data wrangling
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

,Ninth Grade Reading Scores,Tenth Grade Reading Scores,Eleventh Grade Reading Scores,Twelth Grade Reading Scores
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


In [81]:
# establish the bins
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [82]:
#create a copy of the school summar since it has the per student budget, this was done several cells above

In [92]:
# use pd.cut to categorize spending based on the bins
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Budget"], 
                                                             spending_bins, labels = labels, include_lowest=True)

school_spending_df

,School Types,Total Students per School,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Students Passing Math,% Students Passing Reading,% Overall Passing Rate,Spending Ranges (Per Student)
school_name,,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.05,81.03,81.93,81.93,54.64,$585-630
Cabrera High School,Charter,1858,1081356,582.0,83.06,83.98,97.04,97.04,91.33,<$585
Figueroa High School,District,2949,1884411,639.0,76.71,81.16,80.74,80.74,53.20,$630-645
Ford High School,District,2739,1763916,644.0,77.10,80.75,79.30,79.30,54.29,$630-645
Griffin High School,Charter,1468,917500,625.0,83.35,83.82,97.14,97.14,90.60,$585-630
Hernandez High School,District,4635,3022020,652.0,77.29,80.93,80.86,80.86,53.53,$645-680
Holden High School,Charter,427,248087,581.0,83.80,83.81,96.25,96.25,89.23,<$585
Huang High School,District,2917,1910635,655.0,76.63,81.18,81.32,81.32,53.51,$645-680
Johnson High School,District,4761,3094650,650.0,77.07,80.97,81.22,81.22,53.54,$645-680


In [86]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Students Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Students Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing Rate"].mean()

In [94]:
# Assemble into DataFrame
spending_summary = pd.DataFrame({
    "Average Math Score": spending_math_scores,
    "Average Reading Score": spending_reading_scores,
    '% Passing Math': spending_passing_math,
    '% Passing Reading': spending_passing_reading,
    "Overall Passing Rate": overall_passing_spending})

spending_summary.index.name = "Spending Ranges (Per Student)"

# Display results
spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.452500,83.935000,96.610000,96.610000,90.367500
$585-630,81.900000,83.155000,92.717500,92.717500,81.417500
$630-645,78.517500,81.625000,84.392500,84.392500,62.857500
$645-680,76.996667,81.026667,81.133333,81.133333,53.526667


In [95]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = 
per_school_summary

In [ ]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()

In [ ]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = 

# Display results
size_summary

In [ ]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

In [ ]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = 

# Display results
type_summary